# Dear reader:
# The maps in this notebook may not render properly in github.com. Please copy this notebook's url address into this website:

# https://nbviewer.jupyter.org/

# To view this project optimally.

In [2]:
import json
from geopy.distance import geodesic
import requests 
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
print('Libraries imported.')

Libraries imported.


In [42]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium.plugins import MarkerCluster

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

# Capstone project - Where in Portland, Oregon, U.S.A. would be a good location to open a Korean restaurant? 

## Data Science Certificate Capstone with IBM/Coursera: Dylan Amberson, March 5th, 2020

# Introduction: Business Problem

This notebook will give insight into where the best neighborhood to open a restaurant serving Korean food might be in Portland, Oregon. This notebook would be useful to those who are considering open a new restaurant and want to use data to help inform their site selection. 

Portland, Oregon has built a reputation as being a wonderful city for any range of foods. There are ample brew pubs and ice cream shops as well as a dynamic and ecletic food cart scene. Korean food is experiencing a growth in popularity in many big cities in America. According to **Tae Hong** at *chartermedia.com* (https://charactermedia.com/korean-restaurants-continue-growth-in-us-number-5000/):

**"In 2014, the National Restaurant Association ranked Korean food as No. 2 on its list of hottest ethnic cuisines."**

A Korean restaurant is a great opportunity to expand this often overlooked class of food, especially in Portland where Vietnamese soup and Japanese sushi offerings far outweigh Korean kimchi jjigae and marinated galbi.

There are many ways to determine a good site for opening a restaurant. For this notebook I will consider how many people are in each neighborhood (potential customers) and how far that neighborhood is from an existing Korean restaurant.

**In this notebook I will answer the question:**

### Which neighborhood has the most people that are the farthest from an existing Korean restaurant?

# Data:

With our question now framed above, Let us discuss the data we will be using:

    -A list of all the neighborhoods in Portland (https://gis-pdx.opendata.arcgis.com/search?tags=boundary)
    
    -Latitude and Longitude coordinates for each neighborhood. I used this website to ensure correct returns on neighborhood searches. 
      (https://www.gps-latitude-longitude.com/address-to-longitude-latitude-gps-coordinates)
      
    -Population data for each neighborhood from the 2010 census. (https://www.gps-latitude-longitude.com/address-to-longitude-latitude-gps-coordinates)
    
    -Foursquare API will also be used to extract data on existing Korean restaurants in Portland, Oregon.

### Let's start by importing our neighborhood data:
For this notebook's purpose, we will be using the neighborhood's names, as well the area in square feet. There is also the population for each neighborhood from the 2010 census. Because the neighborhoods are all different sizes, I need a density number to express how many people are in each square foot of the neighborhood. Dividing population by neighborhood size give the population density, people per square foot. 

In [17]:
# The code was removed by Watson Studio for sharing.

,OBJECTID,NAME,COMMPLAN,SHARED,COALIT,HORZ_VERT,MAPLABEL,ID,Shape_Length,Shape_Area,Latitude,Longitude,Population 2010 Census,POPULATION DENSITY
0,76,ALAMEDA,,,NECN,HORZ,Alameda,1,8442.289859,3.240808e+06,45.549407,-122.637715,5214.0,0.001609
1,6,ARBOR LODGE,ALBINA,,NPNS,HORZ,Arbor Lodge,2,9466.411504,4.587000e+06,45.572629,-122.689902,6153.0,0.001341
2,60,ARDENWALD-JOHNSON CREEK,,,SEUL,HORZ,Ardenwald-Johnson Creek,3,15320.697117,5.593594e+06,45.455397,-122.629816,4748.0,0.000849
3,71,ARGAY TERRACE,,,EPCO,VERT,Argay Terrace,4,13090.588665,1.091007e+07,45.554502,-122.521303,6006.0,0.000551
4,56,ARLINGTON HEIGHTS,,,NWNW,HORZ,Arlington Heights,5,9286.200340,4.839817e+06,45.521127,-122.711556,718.0,0.000148


## Now to clean up the data:
I will drop some columns I don't need as well as drop some neighborhoods that are not going to be useful to our analysis. 

In [18]:
pdx.drop(['COMMPLAN', 'SHARED', 'HORZ_VERT', 'MAPLABEL', 'ID', 'Shape_Length'],   axis=1, inplace=True)

In [19]:
pdx.rename(columns={'POPULATION DENSITY':'POPULATION_DENSITY'}, inplace=True)

**I know from my data selction time that their are some neighborhoods that are "unclaimed". I will omit these from my analysis because they mostly industrial or transportation land. Not suitable for our restaurant.**

In [20]:
def nans(pdx): return pdx[pdx.isnull().any(axis=1)]

In [21]:
nans(pdx)

,OBJECTID,NAME,COALIT,Shape_Area,Latitude,Longitude,Population 2010 Census,POPULATION_DENSITY
52,45,MC UNCLAIMED #11,UNCLAIMED,3.318922e+06,NaN,NaN,NaN,NaN
53,50,MC UNCLAIMED #13,UNCLAIMED,3.431726e+06,NaN,NaN,NaN,NaN
54,54,MC UNCLAIMED #14,UNCLAIMED,1.593599e+07,NaN,NaN,NaN,NaN
55,18,MC UNCLAIMED #5,UNCLAIMED,1.228032e+05,45.505106,-122.675026,NaN,NaN


In [22]:
pdx.drop(pdx.index[[52,53,54,55]], inplace=True)

In [23]:
pdx.reset_index(inplace=True)

In [24]:
pdx.drop(['index'], axis=1, inplace=True)

### The dataframe for the Portland neighborhoods consists of 94 different neighborhoods. 
Let's look at them on a map:

In [25]:
locations = pdx[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)

94

In [26]:
latitude=45.5051
longitude=-122.5950
pdx_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for point in range(0, len(locationlist)):
    folium.CircleMarker(locationlist[point], radius=6,  color='green', fill=True, fill_color='red', fill_opacity=1).add_to(pdx_map) 
    
pdx_map

NameError: name 'folium' is not defined

In [27]:
pdx

,OBJECTID,NAME,COALIT,Shape_Area,Latitude,Longitude,Population 2010 Census,POPULATION_DENSITY
0,76,ALAMEDA,NECN,3.240808e+06,45.549407,-122.637715,5214.0,0.001609
1,6,ARBOR LODGE,NPNS,4.587000e+06,45.572629,-122.689902,6153.0,0.001341
2,60,ARDENWALD-JOHNSON CREEK,SEUL,5.593594e+06,45.455397,-122.629816,4748.0,0.000849
3,71,ARGAY TERRACE,EPCO,1.091007e+07,45.554502,-122.521303,6006.0,0.000551
4,56,ARLINGTON HEIGHTS,NWNW,4.839817e+06,45.521127,-122.711556,718.0,0.000148
5,51,ARNOLD CREEK,SWNI,5.770166e+06,45.438259,-122.701542,3125.0,0.000542
6,58,ASHCREEK,SWNI,5.149641e+06,45.464293,-122.734997,5719.0,0.001111
7,74,BEAUMONT-WILSHIRE,CNN,3.898812e+06,45.553110,-122.622034,5346.0,0.001371
8,79,BOISE,NECN,2.259639e+06,45.550806,-122.672318,3311.0,0.001465
9,43,BRENTWOOD-DARLINGTON,SEUL,9.229278e+06,45.471180,-122.598773,12994.0,0.001408


# Foursquare data source:

Now I need to find all the Korean restaurants within a 50000 unit radius of the Portland center. To do this I will be using the Foursquare API and searching for "Korean"

In [28]:
# The code was removed by Watson Studio for sharing.

In [29]:
search_query = "Korean"
radius = 50000

**My Foursquare URL. My credentials are hidden**

In [30]:
# The code was removed by Watson Studio for sharing.

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e630b19963d29001ba935f6'},
 'response': {'venues': [{'id': '4ca24baf5720b1f7890228ef',
    'name': 'Korean Twist',
    'location': {'address': 'SW 5th Ave',
     'crossStreet': 'Oak St',
     'lat': 45.52129054586379,
     'lng': -122.67611952674812,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.52129054586379,
       'lng': -122.67611952674812}],
     'distance': 1804,
     'cc': 'US',
     'neighborhood': 'Downtown Portland',
     'city': 'Portland',
     'state': 'OR',
     'country': 'United States',
     'formattedAddress': ['SW 5th Ave (Oak St)',
      'Portland, OR',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d113941735',
      'name': 'Korean Restaurant',
      'pluralName': 'Korean Restaurants',
      'shortName': 'Korean',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/korean_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1583549454',
    'hasPerk': 

In [32]:
venues = results['response']['venues']
venues

dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ca24baf5720b1f7890228ef,SW 5th Ave,...,1804,"[SW 5th Ave (Oak St), Portland, OR, United Sta...","[{'label': 'display', 'lat': 45.52129054586379...",45.521291,-122.676120,Downtown Portland,NaN,OR,Korean Twist,v-1583549454
1,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b956e72f964a52037a234e3,340 SW 5th Ave,...,1759,"[340 SW 5th Ave (at SW Stark St), Portland, OR...","[{'label': 'display', 'lat': 45.52088009286055...",45.520880,-122.676208,NaN,97204,OR,Bulkogi Korean BBQ,v-1583549454
2,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ac7eeb5f964a520abba20e3,1203 NW 23rd Ave,...,3477,"[1203 NW 23rd Ave (Northrup), Portland, OR 972...","[{'label': 'display', 'lat': 45.53142800000000...",45.531428,-122.698993,NaN,97210,OR,Be Won Korean Restaurant,v-1583549455
3,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",1784570,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/korean-twis...,False,5c86bb2d234724003917f6d1,NaN,...,1787,"[Portland, OR 97214, United States]","[{'label': 'display', 'lat': 45.512764, 'lng':...",45.512764,-122.654861,NaN,97214,OR,Korean Twist,v-1583549455
4,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",NaN,NaN,NaN,NaN,NaN,NaN,False,4cdc3df7fc973704d225d405,NaN,...,1771,"[Portland, OR 97205, United States]","[{'label': 'display', 'lat': 45.520474, 'lng':...",45.520474,-122.680882,NaN,97205,OR,Korean Tofu,v-1583549455


In [33]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korean Twist,Korean Restaurant,SW 5th Ave,US,Portland,United States,Oak St,1804,"[SW 5th Ave (Oak St), Portland, OR, United Sta...","[{'label': 'display', 'lat': 45.52129054586379...",45.521291,-122.676120,Downtown Portland,NaN,OR,4ca24baf5720b1f7890228ef
1,Bulkogi Korean BBQ,Food Truck,340 SW 5th Ave,US,Portland,United States,at SW Stark St,1759,"[340 SW 5th Ave (at SW Stark St), Portland, OR...","[{'label': 'display', 'lat': 45.52088009286055...",45.520880,-122.676208,NaN,97204,OR,4b956e72f964a52037a234e3
2,Be Won Korean Restaurant,Korean Restaurant,1203 NW 23rd Ave,US,Portland,United States,Northrup,3477,"[1203 NW 23rd Ave (Northrup), Portland, OR 972...","[{'label': 'display', 'lat': 45.53142800000000...",45.531428,-122.698993,NaN,97210,OR,4ac7eeb5f964a520abba20e3
3,Korean Twist,Food Truck,NaN,US,Portland,United States,NaN,1787,"[Portland, OR 97214, United States]","[{'label': 'display', 'lat': 45.512764, 'lng':...",45.512764,-122.654861,NaN,97214,OR,5c86bb2d234724003917f6d1
4,Korean Tofu,Korean Restaurant,NaN,US,Portland,United States,NaN,1771,"[Portland, OR 97205, United States]","[{'label': 'display', 'lat': 45.520474, 'lng':...",45.520474,-122.680882,NaN,97205,OR,4cdc3df7fc973704d225d405


### This dataframe includes some Korean establishments that are not in Portland, but in surrounding towns and suburbs. I will remove those that are not within the Portland city limits.  

In [34]:
pdx_korean = dataframe_filtered.loc[dataframe_filtered['city'] == 'Portland']
pdx_korean.reset_index(inplace=True)

In [35]:
pdx_korean.drop(['index'],   axis=1, inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### I also see we have some results that are not restaurants or places that serve food, such as churches. We will remove non-food serving places. 

In [36]:
pdx_korean

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korean Twist,Korean Restaurant,SW 5th Ave,US,Portland,United States,Oak St,1804,"[SW 5th Ave (Oak St), Portland, OR, United Sta...","[{'label': 'display', 'lat': 45.52129054586379...",45.521291,-122.676120,Downtown Portland,NaN,OR,4ca24baf5720b1f7890228ef
1,Bulkogi Korean BBQ,Food Truck,340 SW 5th Ave,US,Portland,United States,at SW Stark St,1759,"[340 SW 5th Ave (at SW Stark St), Portland, OR...","[{'label': 'display', 'lat': 45.52088009286055...",45.520880,-122.676208,NaN,97204,OR,4b956e72f964a52037a234e3
2,Be Won Korean Restaurant,Korean Restaurant,1203 NW 23rd Ave,US,Portland,United States,Northrup,3477,"[1203 NW 23rd Ave (Northrup), Portland, OR 972...","[{'label': 'display', 'lat': 45.53142800000000...",45.531428,-122.698993,NaN,97210,OR,4ac7eeb5f964a520abba20e3
3,Korean Twist,Food Truck,NaN,US,Portland,United States,NaN,1787,"[Portland, OR 97214, United States]","[{'label': 'display', 'lat': 45.512764, 'lng':...",45.512764,-122.654861,NaN,97214,OR,5c86bb2d234724003917f6d1
4,Korean Tofu,Korean Restaurant,NaN,US,Portland,United States,NaN,1771,"[Portland, OR 97205, United States]","[{'label': 'display', 'lat': 45.520474, 'lng':...",45.520474,-122.680882,NaN,97205,OR,4cdc3df7fc973704d225d405
5,Korean Food Cart,Korean Restaurant,NaN,US,Portland,United States,SW 3rd & Alder,1764,"[Portland, OR, United States]","[{'label': 'display', 'lat': 45.52058869887465...",45.520589,-122.679824,NaN,NaN,OR,4ca3d2661ee76dcbf4a4fcdd
6,Portland Korean Church,Church,1100 SW 6th Ave,US,Portland,United States,NaN,1255,"[1100 SW 6th Ave, Portland, OR 97204, United S...","[{'label': 'display', 'lat': 45.51589699999999...",45.515897,-122.679639,NaN,97204,OR,4d178f62e190721ec1ce1b21
7,#1 Bento Korean BBQ,Korean Restaurant,SW 10th & Alder St,US,Portland,United States,SW Alder St,1819,"[SW 10th & Alder St (SW Alder St), Portland, O...","[{'label': 'display', 'lat': 45.52078715567763...",45.520787,-122.681544,NaN,97205,OR,4bd361dd41b9ef3b0e0d00e6
8,K-Town Korean BBQ,Restaurant,NaN,US,Portland,United States,NaN,8035,"[Portland, OR 97266, United States]","[{'label': 'display', 'lat': 45.48237626032542...",45.482376,-122.577266,NaN,97266,OR,55becbd3498e6973021c4dcb
9,Josun Korean BBQ,Asian Restaurant,8201 SE Powell Blvd,US,Portland,United States,NaN,7615,"[8201 SE Powell Blvd, Portland, OR 97266, Unit...","[{'label': 'display', 'lat': 45.4977898164142,...",45.497790,-122.577948,NaN,97266,OR,4ed469080aafb9162692314c


In [37]:
pdx_korean.drop(pdx_korean.index[[6,13,16]], inplace=True)

In [38]:
pdx_korean.reset_index(inplace=True)

In [39]:
pdx_korean_res=pdx_korean

In [40]:
pdx_korean_res

,index,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,0,Korean Twist,Korean Restaurant,SW 5th Ave,US,Portland,United States,Oak St,1804,"[SW 5th Ave (Oak St), Portland, OR, United Sta...","[{'label': 'display', 'lat': 45.52129054586379...",45.521291,-122.676120,Downtown Portland,NaN,OR,4ca24baf5720b1f7890228ef
1,1,Bulkogi Korean BBQ,Food Truck,340 SW 5th Ave,US,Portland,United States,at SW Stark St,1759,"[340 SW 5th Ave (at SW Stark St), Portland, OR...","[{'label': 'display', 'lat': 45.52088009286055...",45.520880,-122.676208,NaN,97204,OR,4b956e72f964a52037a234e3
2,2,Be Won Korean Restaurant,Korean Restaurant,1203 NW 23rd Ave,US,Portland,United States,Northrup,3477,"[1203 NW 23rd Ave (Northrup), Portland, OR 972...","[{'label': 'display', 'lat': 45.53142800000000...",45.531428,-122.698993,NaN,97210,OR,4ac7eeb5f964a520abba20e3
3,3,Korean Twist,Food Truck,NaN,US,Portland,United States,NaN,1787,"[Portland, OR 97214, United States]","[{'label': 'display', 'lat': 45.512764, 'lng':...",45.512764,-122.654861,NaN,97214,OR,5c86bb2d234724003917f6d1
4,4,Korean Tofu,Korean Restaurant,NaN,US,Portland,United States,NaN,1771,"[Portland, OR 97205, United States]","[{'label': 'display', 'lat': 45.520474, 'lng':...",45.520474,-122.680882,NaN,97205,OR,4cdc3df7fc973704d225d405
5,5,Korean Food Cart,Korean Restaurant,NaN,US,Portland,United States,SW 3rd & Alder,1764,"[Portland, OR, United States]","[{'label': 'display', 'lat': 45.52058869887465...",45.520589,-122.679824,NaN,NaN,OR,4ca3d2661ee76dcbf4a4fcdd
6,7,#1 Bento Korean BBQ,Korean Restaurant,SW 10th & Alder St,US,Portland,United States,SW Alder St,1819,"[SW 10th & Alder St (SW Alder St), Portland, O...","[{'label': 'display', 'lat': 45.52078715567763...",45.520787,-122.681544,NaN,97205,OR,4bd361dd41b9ef3b0e0d00e6
7,8,K-Town Korean BBQ,Restaurant,NaN,US,Portland,United States,NaN,8035,"[Portland, OR 97266, United States]","[{'label': 'display', 'lat': 45.48237626032542...",45.482376,-122.577266,NaN,97266,OR,55becbd3498e6973021c4dcb
8,9,Josun Korean BBQ,Asian Restaurant,8201 SE Powell Blvd,US,Portland,United States,NaN,7615,"[8201 SE Powell Blvd, Portland, OR 97266, Unit...","[{'label': 'display', 'lat': 45.4977898164142,...",45.497790,-122.577948,NaN,97266,OR,4ed469080aafb9162692314c
9,10,Seoul Korean Restaurant,Korean Restaurant,11211 SE 82nd Ave,US,Portland,United States,NaN,10176,"[11211 SE 82nd Ave, Portland, OR 97086, United...","[{'label': 'display', 'lat': 45.442494, 'lng':...",45.442494,-122.580005,NaN,97086,OR,593df8cb6cf01a13d3d57f37


In [43]:
pdx_korean_res.shape

(19, 17)

## There are 19 restaurants that are showing "Korean" in the Foursquare API. 
As can be seen on the map below. They are not equally distributed throughout the neighborhoods. Perhaps we can use data to find a neighborhood with many people that is not near an existing restaurant. 

In [45]:
latitude=45.5051
longitude=-122.5950
korean_map = folium.Map(location=[latitude, longitude], zoom_start=12)


for lat, lng, label in zip(pdx_korean_res.lat, pdx_korean_res.lng,  pdx_korean_res.name):
    folium.features.Marker(
        [lat, lng],
        popup=label,
       ).add_to(korean_map)
    
folium.features.Marker(
        [45.512764, -122.654861],
        popup="Korean Twist",
       ).add_to(korean_map)
korean_map

# Methodology:
With our neighborhood and restaurant data now saved in dataframes, it is time to discuss how we will select the best neighborhood for a new Korean restaurant.

The next step is to discover how far (in miles) each restaurant is from each neighborhood. I will use neighborhood latitude and longitude and well as latitude and longitude for each restaurant. 

- First I will create a smaller dataframe from the Foursquare data with only names and coordinates.

- After that I will find the minimum distance each neighborhood is to a Korean restaurant. This will help illustrate competition and the overall servicing of the Korean restaurant market.

- Then I will create a function of how under served a neighborhood is by inputting distance from existing restaurants and the population density of the neighborhood. There are some more remote neighborhoods that are very far from Korean food but looking at the population numbers, there are many fewer people there than other neighborhoods. It would not be advisable to avoid competition and in the process avoid people all together. A note, I will be multiplying distance by population density. It is possible in further analysis to weigh these two inputs differently. Maybe more people in a neighborhood is more important to distance to a competitors.  

- Finally I will find the three neighborhoods that have the most residents that are farthest from an existing Korean restaurant. 

In [46]:
koreangps= pdx_korean_res[['name','lat','lng']].copy()

In [47]:
koreangps

,name,lat,lng
0,Korean Twist,45.521291,-122.676120
1,Bulkogi Korean BBQ,45.520880,-122.676208
2,Be Won Korean Restaurant,45.531428,-122.698993
3,Korean Twist,45.512764,-122.654861
4,Korean Tofu,45.520474,-122.680882
5,Korean Food Cart,45.520589,-122.679824
6,#1 Bento Korean BBQ,45.520787,-122.681544
7,K-Town Korean BBQ,45.482376,-122.577266
8,Josun Korean BBQ,45.497790,-122.577948
9,Seoul Korean Restaurant,45.442494,-122.580005


**Now I need a matrix of distances.** The matrix needs to show how far each neighborhood is from each restaurant. This matrix will be 94 rows (one for each neighborhood) and 54 columns (each restaurant needs NAME, LATITUDE, and LONGITUDE.

In [48]:
k0 = ['Korean Twist 1']
k0lat= [45.521291]
k0lng= [-122.676120]

k1= ['Bulkogi Korean BBQ']
k1lat= [45.520880]
k1lng= [-122.676208]

k2= ['Be Won Korean Restaurant']
k2lat=[45.531428]
k2lng=[-122.698993]

k3= ['Korean Food Cart 1']
k3lat=[45.520589]
k3lng=[-122.679824]

k4= ['Korean Tofu']
k4lat=[45.520474]
k4lng=[-122.680882]

k5= ['Korean Twist 2']
k5lat=[45.512764]
k5lng=[-122.654861]

k6= ['K-Town Korean BBQ']
k6lat=[45.482376]
k6lng=[-122.577266]

k7= ['#1 Bento Korean BBQ']
k7lat=[45.520787]
k7lng=[-122.681544]

k8= ['Josun Korean BBQ']
k8lat=[45.497790]
k8lng=[-122.577948]

k9= ['Seoul Korean Restaurant']
k9lat=[45.442494]
k9lng=[-122.580005]

k10= ['Sari Korean Restaurant']
k10lat=[45.529003]
k10lng=[-122.637927]

k11= ['수라 Korean Restaurant']
k11lat=[45.525406]
k11lng=[-122.698562]

k12= ['New Seoul Garden Yakiniku Korean BBQ']
k12lat=[45.483908]
k12lng=[-122.716922]

k13= ['Kkoki Korean Bbq']
k13lat=[45.498930]
k13lng=[-122.579160]

k14= ['K-Town Korean BBQ']
k14lat=[45.482361]
k14lng=[-122.577237]

k15= ['Namu Korean Hawaiian BBQ']
k15lat=[45.533991]
k15lng=[-122.601121]


k16 =["Chungdam Korean Fusion"]
k16lat=[45.497526]
k16lng=[-122.581199]

k17 = ['Korean Twist 3']
k17lat = [45.589347]
k17lng=[-122.593755]

k18 = ['Namu Korean & Hawaiian']
k18lat=[45.519036]
k18lng=[-122.584679]       

In [49]:
df = pd.DataFrame(list(zip(k0, k0lat, k0lng, k1, k1lat, k1lng, k2, k2lat, k2lng, k3, k3lat, k3lng, k4, k4lat, k4lng, k5, k5lat, k5lng, k6, k6lat, k6lng, k7, k7lat, k7lng, k8, k8lat, k8lng, k9, k9lat, k9lng, k10, k10lat, k10lng, k11, k11lat, k11lng, k12, k12lat, k12lng, k13, k13lat, k13lng, k14, k14lat, k14lng, k15, k15lat, k15lng, k16, k16lat, k16lng, k17, k17lat, k17lng, k18, k18lat, k18lng)), 
               columns =['RESTURANT_0', 'lat_0', 'lng_0', 'RESTURANT_1', 'lat_1', 'lng_1', 'RESTURANT_2', 'lat_2', 'lng_2', 'RESTURANT_3', 'lat_3', 'lng_3', 'RESTURANT_4', 'lat_4', 'lng_4', 'RESTURANT_5', 'lat_5', 'lng_5', 'RESTURANT_6', 'lat_6', 'lng_6', 'RESTURANT_7', 'lat_7', 'lng_7', 'RESTURANT_8', 'lat_8', 'lng_8', 'RESTURANT_9', 'lat_9', 'lng_9', 'RESTURANT_10', 'lat_10', 'lng_10', 'RESTURANT_11', 'lat_11', 'lng_11', 'RESTURANT_12', 'lat_12', 'lng_12', 'RESTURANT_13', 'lat_13', 'lng_13', 'RESTURANT_14', 'lat_14', 'lng_14', 'RESTURANT_15', 'lat_15', 'lng_15', 'RESTURANT_16', 'lat_16', 'lng_16', 'RESTURANT_17', 'lat_17', 'lng_17', 'RESTURANT_18', 'lat_18', 'lng_18']) 
df

,RESTURANT_0,lat_0,lng_0,RESTURANT_1,lat_1,lng_1,RESTURANT_2,lat_2,lng_2,RESTURANT_3,...,lng_15,RESTURANT_16,lat_16,lng_16,RESTURANT_17,lat_17,lng_17,RESTURANT_18,lat_18,lng_18
0,Korean Twist 1,45.521291,-122.67612,Bulkogi Korean BBQ,45.52088,-122.676208,Be Won Korean Restaurant,45.531428,-122.698993,Korean Food Cart 1,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679


#### Now to multiple for each neighborhood.

In [50]:
df = pd.concat([df]*94, ignore_index=True)

In [51]:
df.shape

(94, 57)

#### ...And add to my neighborhood dataframe.

In [52]:
pdx= pd.concat([pdx, df], axis=1)

In [53]:
pdx

,OBJECTID,NAME,COALIT,Shape_Area,Latitude,Longitude,Population 2010 Census,POPULATION_DENSITY,RESTURANT_0,lat_0,...,lng_15,RESTURANT_16,lat_16,lng_16,RESTURANT_17,lat_17,lng_17,RESTURANT_18,lat_18,lng_18
0,76,ALAMEDA,NECN,3.240808e+06,45.549407,-122.637715,5214.0,0.001609,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
1,6,ARBOR LODGE,NPNS,4.587000e+06,45.572629,-122.689902,6153.0,0.001341,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
2,60,ARDENWALD-JOHNSON CREEK,SEUL,5.593594e+06,45.455397,-122.629816,4748.0,0.000849,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
3,71,ARGAY TERRACE,EPCO,1.091007e+07,45.554502,-122.521303,6006.0,0.000551,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
4,56,ARLINGTON HEIGHTS,NWNW,4.839817e+06,45.521127,-122.711556,718.0,0.000148,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
5,51,ARNOLD CREEK,SWNI,5.770166e+06,45.438259,-122.701542,3125.0,0.000542,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
6,58,ASHCREEK,SWNI,5.149641e+06,45.464293,-122.734997,5719.0,0.001111,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
7,74,BEAUMONT-WILSHIRE,CNN,3.898812e+06,45.553110,-122.622034,5346.0,0.001371,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
8,79,BOISE,NECN,2.259639e+06,45.550806,-122.672318,3311.0,0.001465,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679
9,43,BRENTWOOD-DARLINGTON,SEUL,9.229278e+06,45.471180,-122.598773,12994.0,0.001408,Korean Twist 1,45.521291,...,-122.601121,Chungdam Korean Fusion,45.497526,-122.581199,Korean Twist 3,45.589347,-122.593755,Namu Korean & Hawaiian,45.519036,-122.584679


 * **I was having trouble looping the neighborhood gps coordiantes at the same time as the restaurant coordiantes using geodisc from geopy. I made a loop for each restaurant.** 

In [54]:
#for restaurant in the 0 index
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_0"] = geodesic((row['Latitude'], row['Longitude']), ((45.521291, -122.601121))).mi
#and restaurant in the 1 index    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_1"] = geodesic((row['Latitude'], row['Longitude']), ((45.520880, -122.676208))).mi
#and so on...
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_2"] = geodesic((row['Latitude'], row['Longitude']), ((45.531428, -122.698993))).mi

for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_3"] = geodesic((row['Latitude'], row['Longitude']), ((45.520589, -122.679824))).mi

for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_4"] = geodesic((row['Latitude'], row['Longitude']), ((45.520474, -122.680882))).mi

for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_5"] = geodesic((row['Latitude'], row['Longitude']), ((45.512764, -122.654861))).mi

for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_6"] = geodesic((row['Latitude'], row['Longitude']), ((45.482376, -122.577266))).mi

for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_7"] = geodesic((row['Latitude'], row['Longitude']), ((45.520787, -122.577948))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_8"] = geodesic((row['Latitude'], row['Longitude']), ((45.497790, -122.654861))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_9"] = geodesic((row['Latitude'], row['Longitude']), ((45.442494, -122.580005))).mi

for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_10"] = geodesic((row['Latitude'], row['Longitude']), ((45.529003, -122.637927))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_11"] = geodesic((row['Latitude'], row['Longitude']), ((45.525406, -122.698562))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_12"] = geodesic((row['Latitude'], row['Longitude']), ((45.483908, -122.716922))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_13"] = geodesic((row['Latitude'], row['Longitude']), ((45.498930, -122.579160))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_14"] = geodesic((row['Latitude'], row['Longitude']), ((45.482361, -122.577237))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_15"] = geodesic((row['Latitude'], row['Longitude']), ((45.533991, -122.601121))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_16"] = geodesic((row['Latitude'], row['Longitude']), ((45.497526, -122.581199))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_17"] = geodesic((row['Latitude'], row['Longitude']), ((45.589347, -122.593755))).mi
    
for label, row in pdx.iterrows():
    pdx.loc[label, "DISTANCE_18"] = geodesic((row['Latitude'], row['Longitude']), ((45.519036,-122.584679))).mi

In [55]:
D_0 = pd.DataFrame(pdx.DISTANCE_0)
D_1 = pd.DataFrame(pdx.DISTANCE_1)
D_2 = pd.DataFrame(pdx.DISTANCE_2)
D_3 = pd.DataFrame(pdx.DISTANCE_3)
D_4 = pd.DataFrame(pdx.DISTANCE_4)
D_5 = pd.DataFrame(pdx.DISTANCE_5)
D_6 = pd.DataFrame(pdx.DISTANCE_6)
D_7 = pd.DataFrame(pdx.DISTANCE_7)
D_8 = pd.DataFrame(pdx.DISTANCE_8)
D_9 = pd.DataFrame(pdx.DISTANCE_9)
D_10 = pd.DataFrame(pdx.DISTANCE_10)
D_11= pd.DataFrame(pdx.DISTANCE_11)
D_12 = pd.DataFrame(pdx.DISTANCE_12)
D_13 = pd.DataFrame(pdx.DISTANCE_13)
D_14 = pd.DataFrame(pdx.DISTANCE_14)
D_15 = pd.DataFrame(pdx.DISTANCE_15)
D_16 = pd.DataFrame(pdx.DISTANCE_16)
D_17 = pd.DataFrame(pdx.DISTANCE_17)
D_18 = pd.DataFrame(pdx.DISTANCE_18)

#### Below is the matrix of distances from neighborhoods to restaurants.

In [56]:
df_dist = pd.concat([D_0, D_1, D_2, D_3, D_4, D_5, D_6, D_7, D_8, D_9, D_10, D_11, D_12, D_13, D_14, D_15, D_16, D_17, D_18], axis= 1, sort=False)

df_dist

,DISTANCE_0,DISTANCE_1,DISTANCE_2,DISTANCE_3,DISTANCE_4,DISTANCE_5,DISTANCE_6,DISTANCE_7,DISTANCE_8,DISTANCE_9,DISTANCE_10,DISTANCE_11,DISTANCE_12,DISTANCE_13,DISTANCE_14,DISTANCE_15,DISTANCE_16,DISTANCE_17,DISTANCE_18
0,2.631481,2.715076,3.222644,2.852686,2.895174,2.663923,5.481142,3.510156,3.660580,7.897541,1.409151,3.386460,5.937119,4.497982,5.482771,2.070558,4.512706,3.486389,3.320460
1,5.579397,3.635076,2.879361,3.627040,3.628359,4.470372,8.291141,6.506223,5.441051,10.452111,3.929024,3.288216,6.265922,7.402258,8.292849,5.067106,7.398413,4.803515,6.306358
2,4.759327,5.052633,6.233732,5.115689,5.133339,4.144372,3.160975,5.171139,3.170459,2.580027,5.098479,5.875880,4.668151,3.885281,3.161504,5.603789,3.747552,9.414832,4.911481
3,4.501843,7.868372,8.768979,8.042066,8.093512,7.094368,5.673865,3.602654,7.574438,8.243942,5.927308,8.834082,10.675286,4.755695,5.674101,4.124224,4.892467,4.259136,3.931922
4,5.361292,1.716128,0.937008,1.540949,1.489822,2.812515,7.049342,6.486329,3.190005,8.386424,3.615367,0.696584,2.583527,6.608930,7.051039,5.433792,6.536242,7.406938,6.161304
5,7.528787,5.837036,6.435440,5.782775,5.765797,5.623008,6.764662,8.278728,4.695462,5.915925,6.987336,6.020107,3.239902,7.274610,6.765455,8.216146,7.137864,11.673080,7.959549
6,7.601141,4.839981,4.955137,4.721927,4.686377,5.133959,7.764925,8.567946,4.528551,7.682364,6.495952,4.576458,1.614383,7.939615,7.766151,8.089615,7.816288,11.026907,8.222012
7,2.420519,3.444884,4.023721,3.593157,3.638297,3.209708,5.346628,3.092086,4.139443,7.907167,1.834827,4.177850,6.637922,4.281563,5.348147,1.665329,4.320264,2.853809,2.970582
8,4.011876,2.075313,1.861920,2.118364,2.135585,2.760461,6.605309,5.027525,3.758102,8.720802,2.247857,2.167782,5.102347,5.769363,6.607035,3.645026,5.753672,4.648214,4.786098
9,3.462550,5.091662,6.403146,5.209550,5.243330,3.958417,1.299880,3.572053,3.286476,2.180960,4.422888,6.124678,5.807116,2.140186,1.300398,4.339238,2.009789,8.164309,3.375088


### I now need to find which of these 19 restaurants is the nearest to each of the 94 neighborhoods. 

In [58]:
dfobj1 = df_dist
min_dist = dfobj1.min(axis=1)

In [59]:
min_dist_neigh = pd.DataFrame(min_dist)

In [60]:
min_dist_neigh.rename(columns={0:'min_dist_kres_mi'}, inplace=True)

### ...And add to the neighborhood dataframe.

In [61]:
pdx_final = pd.concat([pdx, min_dist_neigh ], axis= 1, sort=False)

In [62]:
pdx_final

,OBJECTID,NAME,COALIT,Shape_Area,Latitude,Longitude,Population 2010 Census,POPULATION_DENSITY,RESTURANT_0,lat_0,...,DISTANCE_10,DISTANCE_11,DISTANCE_12,DISTANCE_13,DISTANCE_14,DISTANCE_15,DISTANCE_16,DISTANCE_17,DISTANCE_18,min_dist_kres_mi
0,76,ALAMEDA,NECN,3.240808e+06,45.549407,-122.637715,5214.0,0.001609,Korean Twist 1,45.521291,...,1.409151,3.386460,5.937119,4.497982,5.482771,2.070558,4.512706,3.486389,3.320460,1.409151
1,6,ARBOR LODGE,NPNS,4.587000e+06,45.572629,-122.689902,6153.0,0.001341,Korean Twist 1,45.521291,...,3.929024,3.288216,6.265922,7.402258,8.292849,5.067106,7.398413,4.803515,6.306358,2.879361
2,60,ARDENWALD-JOHNSON CREEK,SEUL,5.593594e+06,45.455397,-122.629816,4748.0,0.000849,Korean Twist 1,45.521291,...,5.098479,5.875880,4.668151,3.885281,3.161504,5.603789,3.747552,9.414832,4.911481,2.580027
3,71,ARGAY TERRACE,EPCO,1.091007e+07,45.554502,-122.521303,6006.0,0.000551,Korean Twist 1,45.521291,...,5.927308,8.834082,10.675286,4.755695,5.674101,4.124224,4.892467,4.259136,3.931922,3.602654
4,56,ARLINGTON HEIGHTS,NWNW,4.839817e+06,45.521127,-122.711556,718.0,0.000148,Korean Twist 1,45.521291,...,3.615367,0.696584,2.583527,6.608930,7.051039,5.433792,6.536242,7.406938,6.161304,0.696584
5,51,ARNOLD CREEK,SWNI,5.770166e+06,45.438259,-122.701542,3125.0,0.000542,Korean Twist 1,45.521291,...,6.987336,6.020107,3.239902,7.274610,6.765455,8.216146,7.137864,11.673080,7.959549,3.239902
6,58,ASHCREEK,SWNI,5.149641e+06,45.464293,-122.734997,5719.0,0.001111,Korean Twist 1,45.521291,...,6.495952,4.576458,1.614383,7.939615,7.766151,8.089615,7.816288,11.026907,8.222012,1.614383
7,74,BEAUMONT-WILSHIRE,CNN,3.898812e+06,45.553110,-122.622034,5346.0,0.001371,Korean Twist 1,45.521291,...,1.834827,4.177850,6.637922,4.281563,5.348147,1.665329,4.320264,2.853809,2.970582,1.665329
8,79,BOISE,NECN,2.259639e+06,45.550806,-122.672318,3311.0,0.001465,Korean Twist 1,45.521291,...,2.247857,2.167782,5.102347,5.769363,6.607035,3.645026,5.753672,4.648214,4.786098,1.861920
9,43,BRENTWOOD-DARLINGTON,SEUL,9.229278e+06,45.471180,-122.598773,12994.0,0.001408,Korean Twist 1,45.521291,...,4.422888,6.124678,5.807116,2.140186,1.300398,4.339238,2.009789,8.164309,3.375088,1.299880


# Analysis:
Now having the minimum distance from each neighborhood to the closest Korean restaurant, I can multiply the distance by the population density of each neighborhood.
I am going to use my final 'Restaurant demand' array as a function of distance to the nearest restaurant and how densely populated the neighborhood is.

In [63]:
pdx_final['resturant_demand'] = pdx_final['POPULATION_DENSITY'] * pdx_final['min_dist_kres_mi']

#### Now to find the three best neighborhoods. The three that have the most people the fartehst distance from Korean food. 

In [64]:
three_best = pdx_final.nlargest(3, 'resturant_demand')

In [65]:
three_best[['NAME', 'Population 2010 Census', 'POPULATION_DENSITY', 'min_dist_kres_mi', 'resturant_demand' ]]

,NAME,Population 2010 Census,POPULATION_DENSITY,min_dist_kres_mi,resturant_demand
27,GLENFAIR,3417.0,0.002041,3.552546,0.007251
68,PORTSMOUTH,9789.0,0.001711,3.847780,0.006582
15,CENTENNIAL,23662.0,0.001375,4.023354,0.005531


# I now have three neighborhoods to begin looking into as good choices to open a Korean restaurant where the new restaurant would be far from competition but also have many possible customers in my neighborhood.

In [66]:
folium.features.CircleMarker(
        [45.523472, -122.504869],
        radius=13,
        popup="#1 Glenfair - Restaurant Demand = 0.007251",
        fill=True,
        color='black',
        fill_color='orange',
        fill_opacity=0.6,
       ).add_to(korean_map)

folium.features.CircleMarker(
        [45.585500, -122.718116],
        radius=13,
        popup="#2 Portsmouth - Restaurant Demand = 0.006582",
        fill=True,
        color='green',
        fill_color='yellow',
        fill_opacity=0.6,
       ).add_to(korean_map)

folium.features.CircleMarker(
        [45.504598, -122.496706],
        radius=13,
        popup="#3 Centennial - Restaurant Demand = 0.005531",
        fill=True,
        color='darkgreen',
        fill_color='white',
        fill_opacity=0.6,
       ).add_to(korean_map)



In [67]:
korean_map

# Three Worst Neighborhoods:
It would also be interesting to find the three worst neighborhoods to scout for a restaurant. To do this we can find the n-smallest (n=3) of our restaurant demand feature.

In [68]:
three_worst = pdx_final.nsmallest(3, 'resturant_demand')

In [69]:
three_worst[['NAME', 'Population 2010 Census', 'POPULATION_DENSITY', 'min_dist_kres_mi', 'resturant_demand' ]]

,NAME,Population 2010 Census,POPULATION_DENSITY,min_dist_kres_mi,resturant_demand
4,ARLINGTON HEIGHTS,718.0,0.000148,0.696584,0.000103
46,LINNTON,941.0,0.000022,6.279025,0.000136
84,SUNDERLAND ASSOCIATION OF NEIGHBORS,718.0,0.000077,1.952946,0.000150


Our first worst neighborhood **Arlington Heights** we can see it is less than a mile from an existing Korean restaurant while also having very low population density. It is clearly not a good choice. Interestingly when we plot these neighborhoods we see **Linnton** is very near the enormous Forest Park, creating very low population density. And the other, **Sunderland** is the area around Portland International Airport. Another neighborhood without many restaurants; or foot traffic for that matter. 

In [70]:
latitude=45.5051
longitude=-122.5750
worst = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(pdx_korean_res.lat, pdx_korean_res.lng,  pdx_korean_res.name):
    folium.features.Marker(
        [lat, lng],
        popup=label,
       ).add_to(worst)
    
folium.features.CircleMarker(
        [45.521127, -122.711556],
        radius=13,
        popup="#1 Worst - Arlington Heights",
        fill=True,
        color='black',
        fill_color='black',
        fill_opacity=0.6,
       ).add_to(worst)

folium.features.CircleMarker(
        [45.599793, -122.784327],
        radius=13,
        popup="#2 Worst - Linnton",
        fill=True,
        color='black',
        fill_color='grey',
        fill_opacity=0.6,
       ).add_to(worst)

folium.features.CircleMarker(
        [45.554919, -122.574055],
        radius=13,
        popup="#3 Worst - Sunderland",
        fill=True,
        color='brown',
        fill_color='grey',
        fill_opacity=0.6,
       ).add_to(worst)

    
    
    
   

worst

# Results and Discussion:
We now have **three neighborhoods** that are **good candidates for a new Korean restaurant.** They are **Glenfair, Portsmouth, and Centennial**. 

Now is also a good opportunity to discuss how this analysis could be further refined. In this notebook all residents of the city were treated as equal. This is not the case with restaurant tastes and preferences. Further marketing research could be done to find which demographics would most like to eat Korean food, find where they live in the city and further weigh that in the consideration.

Also thought needs to be given on how best to weigh distance against population. Korean food is a special meal to most. Some people may not be bothered by traveling to a restaurant that they enjoy but do not visit every week. Consumer attitudes on travel could also weigh how distance vs. population is concerned. 

Furthermore with population, not all demand is centered around where people sleep. Where people work and eat lunch can also change the demand for a certain food. In the above map we see many venues in downtown Portland. While many people live there, there is also a lot of demand from "the lunch crowd". 

One last note, restaurants and food carts/trucks are not equal. This is important to consider in a city such as Portland with a large mobile food sector. Building a sit down restaurant near a food truck is not equal to building near another full scale restaurant. 

The author would also like to **acknowledge** the assistance of **Carmen Piekarski at the City of Portland** for help with the units of measurement for each neighborhood area.
